In [1]:
import os 
import sys 
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

from train import generate_training_examples

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import optuna.visualization as vis
import plotly.io as pio

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from abc import ABC, abstractmethod

In [2]:
from model_baseline import SNV2LinFreqs

In [3]:
device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
print(f"Using {device} device")

n = 10000
known_freqs_file = f'known_freqs_{n}.npy'
snv_freqs_file = f'snv_freqs_{n}.npy'

Using mps device


In [4]:
X=np.load(known_freqs_file)
y=np.load(snv_freqs_file)

In [5]:
known_freqs=torch.from_numpy(X)
snv_freqs=torch.from_numpy(y)

In [6]:
known_freqs = known_freqs.to(device)
snv_freqs = snv_freqs.to(device)
dataset = TensorDataset(snv_freqs, known_freqs)

total_size = len(dataset)
test_size = total_size // 10  # 10% for test
validation_size = total_size // 10  # 10% for validation
train_size = total_size - test_size - validation_size  # 80% train

# Split the dataset
train_dataset, validation_dataset, test_dataset = random_split(
        dataset, [train_size, validation_size, test_size]
    )

# verify sizes
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 8000
Validation set size: 1000
Test set size: 1000


In [7]:
batch_size = 1000  # Arbitrarily chosen
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
input_size = snv_freqs.shape[1]
output_size = known_freqs.shape[1]

In [9]:
config = {
        'num_hidden_layers': (1, 5),
        'hidden_layer_size': (16, 256),
        'activation': ['ReLU', 'Tanh', 'Sigmoid'],
        'dropout': (0, 0.5),
        'lr': (1e-5, 1e-1),
        'weight_decay': (1e-5, 1e-1),
        'epochs': 500,
        'patience': 10, 
        'batch_size': (16, 128)
    }

model = SNV2LinFreqs(train_loader, test_loader, input_size, output_size, device)

# Perform optimization
best_params = model.optimize(n_trials=100, config=config, pruner=True)

print('Best parameters:', best_params)

model.plot_trials(params.prefix)

# Update the data
model.update_data(
        torch.cat((x_train, x_test)),
        torch.cat((y_train, y_test)),
        x_validation,
        y_validation
    )
    

# now use model.best_params (set by optimise) to train model on full dataset
val_loss =model.fit(
        params=model.best_params, 
        epochs=config["epochs"], 
        patience=config["patience"])
print(val_loss)

[I 2024-07-16 02:42:41,335] A new study created in memory with name: no-name-b295fc7b-3218-49ca-818f-bd61a36f0f4b


Using mps device
Using mps device


[I 2024-07-16 02:46:55,236] Trial 0 finished with value: 3.236529300920665e-05 and parameters: {'num_hidden_layers': 2, 'hidden_layer_size': 243, 'activation': 'ReLU', 'dropout': 0.04621648713311238, 'weight_decay': 0.0006956577333381845, 'lr': 7.365105603096945e-05}. Best is trial 0 with value: 3.236529300920665e-05.


Using mps device


[I 2024-07-16 02:48:16,380] Trial 1 finished with value: 2.59343705693027e-05 and parameters: {'num_hidden_layers': 2, 'hidden_layer_size': 228, 'activation': 'Tanh', 'dropout': 0.1719773785628511, 'weight_decay': 0.04242003406295808, 'lr': 0.0022479646739231507}. Best is trial 1 with value: 2.59343705693027e-05.


...

Using mps device


[I 2024-07-16 08:46:01,779] Trial 27 finished with value: 3.0172754122759216e-05 and parameters: {'num_hidden_layers': 3, 'hidden_layer_size': 162, 'activation': 'Sigmoid', 'dropout': 0.3217693446123187, 'weight_decay': 0.059805836078675505, 'lr': 0.0018046900389192164}. Best is trial 15 with value: 2.5874742277665064e-05.


Using mps device


[W 2024-07-16 08:49:53,484] Trial 28 failed with parameters: {'num_hidden_layers': 5, 'hidden_layer_size': 141, 'activation': 'Sigmoid', 'dropout': 0.3728392089807381, 'weight_decay': 0.07979202762769191, 'lr': 2.4634665777482832e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/miniconda3/envs/ww/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 273, in <lambda>
    self.study.optimize(lambda trial: self._objective(trial, config), n_trials=n_trials)
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 216, in _objective
    train_losses, val_losses = model.fit(
  File "/Users/DannyPang/Desktop/bioinfmsc/diss/model_baseline.py", line 87, in fit
    train_loss += loss.item()
KeyboardInterrupt
[W 2024-07-16 08:49:53,487] Trial 28 failed with value None.


KeyboardInterrupt: 